Este colab fue desarrollado por Arnold Charry Armero.

# Optimización de Portafolio

Se desea minimizar el riesgo de una inversión cumpliendo con las restricciones de que no se debe sobrepasar el tope del presupuesto y se debe tener un rendimiento mínimo de la inversión. Realice un modelo de programación entera que haga lo mencionado.

## Conjuntos

$ i = \textrm{Conjunto de acciones por invertir, indexado por} \:  i$

## Parámetros

$ \sigma_{i}^{2} = \textrm{Varianza del invertir en la acción } i $

$ p_{i} = \textrm{Precio de inversión de la acción }i $

$ u_{i} = \textrm{Precio de venta de la acción } i $

$ B = \textrm{Presupuesto total} $

$ R = \textrm{Rendimiento esperado de la inversión} $

## Variables de decisión

$ x_{i} = \textrm{cantidad invertida en la acción } i $

## Función Objetivo

 $$ \text{min}\: \sum_{i=1}^{n}\sigma_{i}^{2}x_{i}^{2} $$

## Restricciones

No se puede rebasar el presupuesto total con la suma invertida por activo,

$$ \sum_{i=1}^{n}p_{i}x_{i} \leq B $$



Se debe tener un rendimiento mínimo por activo invertido,

$$ \sum_{i=1}^{n}u_{i}x_{i} \geq R $$


La cantidad invertida por activo debe ser continua,

$$ x_{i} \geq 0, ∀i$$

In [ ]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 82.2 MB/s eta 0:00:00


In [ ]:
from gurobipy import *
import numpy as np
import pandas as pd

In [ ]:
# Conjuntos
inversiones = range(3)

In [ ]:
# Parámetros
sigma = [100, 1600, 100]

P = [50, 40, 25]

U = [55, 50, 20]

B = 100000

R = 115000

In [ ]:
# Crear la instancia del modelo
model = Model("Minimizar_Riesgo")

# Inicialización de las variables de decisión
X = model.addVars(inversiones, vtype=GRB.CONTINUOUS, lb=0, name="X")

# Agregar la función objetivo
model.setObjective(quicksum(sigma[i] * X[i]**2 for i in inversiones), GRB.MINIMIZE)

# Agregar las restricciones

model.addConstr(quicksum(P[i]*X[i] for i in range(3)) <= B, "Presupuesto")

model.addConstr(quicksum(U[i]*X[i] for i in range(3)) >= R, "Rendimiento")

# Resolver el problema
model.optimize()

# -----------------------------
# Resultados
# -----------------------------
if model.Status == GRB.OPTIMAL:
    print("Solución óptima:")

# Resultados
for var in model.getVars():
  print(var.VarName, "=", var.X)

print(f"\nGanancia esperada = {round(sum(U[i] * X[i].x for i in inversiones), 2)}")

print(f"\nGasto Total = {round(sum(P[i] * X[i].x for i in inversiones), 2)}")

print(f"\nFunción Objetivo = {model.objVal:.2f}")

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0x84ab935d
Model has 3 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e+01, 6e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+02, 3e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+05, 1e+05]
Presolve time: 0.01s
Presolved: 2 rows, 3 columns, 6 nonzeros
Presolved model has 3 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.000e+00
 Factor NZ  : 3.000e+00
 Factor Ops : 5.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.03097694e+09 -2.03097694e+09  2.34e+0

In [ ]:
print("Matriz Q del modelo (coeficientes cuadráticos)")
Q = model.getQ()

# Extraer datos de la matriz dispersa
rows, cols, vals = Q.row, Q.col, Q.data

# Crear DataFrame para visualizarla mejor
df_Q = pd.DataFrame({
    "i": rows,
    "j": cols,
    "Valor": vals
})

print("\nCoeficientes no nulos de Q:")
print(df_Q)

# Convertir a matriz densa (NumPy)
Q_dense = np.zeros((3, 3))
for i, j, val in zip(rows, cols, vals):
    Q_dense[i, j] = val

print("\nMatriz Q completa (densa):")
print(Q_dense)

Matriz Q del modelo (coeficientes cuadráticos)

Coeficientes no nulos de Q:
   i  j      Valor
0  0  0    10000.0
1  1  1  2560000.0
2  2  2    10000.0

Matriz Q completa (densa):
[[  10000.       0.       0.]
 [      0. 2560000.       0.]
 [      0.       0.   10000.]]
